In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

In [2]:
dic_dfs_pre = pd.read_pickle('data_pre.pkl')
print(dic_dfs_pre.keys())

dict_keys(['target', 'features', 'others', 'features_vt', 'vt', 'vt_result'])


In [ ]:
# 型別轉換(去更新前檔)
# y = dic_dfs_pre['target'].copy()
# X = dic_dfs_pre['features_vt'].copy()

# cols_to_int = X.select_dtypes(include='bool').columns
# X[cols_to_int] = X[cols_to_int].astype('int')

### Logistic Regression (with L1 penalty) 

In [3]:
# train, test
X_train = dic_dfs_pre['features_vt'][dic_dfs_pre['others']['Group'] == 'train']
X_test = dic_dfs_pre['features_vt'][dic_dfs_pre['others']['Group'] == 'test']
y_train = dic_dfs_pre['target'][dic_dfs_pre['others']['Group'] == 'train']
y_test = dic_dfs_pre['target'][dic_dfs_pre['others']['Group'] == 'test']


In [4]:
lgs = LogisticRegression(penalty='l1', solver='liblinear')
lgs.fit(X=X_train, y=y_train)

LogisticRegression(penalty='l1', solver='liblinear')

In [5]:
# 違規案件預測結果
for i, v in enumerate(lgs.predict_proba(X=X_test)): 
    real = y_test.iloc[i]
    if real: 
        print(f'第 {i} 個樣本為 {real} ，預測機率為 {v[1]}')

# y_test

第 95 個樣本為 True ，預測機率為 0.0008893227651202786
第 591 個樣本為 True ，預測機率為 0.0018711204803484333
第 640 個樣本為 True ，預測機率為 0.005608845331008317
第 688 個樣本為 True ，預測機率為 0.0011934504747227487
第 785 個樣本為 True ，預測機率為 0.010132123545469196
第 889 個樣本為 True ，預測機率為 0.02358990080712836


In [6]:
# 採用特徵
X_train.columns[lgs.coef_.flatten() > 0]

Index(['營業縣市_台東縣', '營業縣市_宜蘭縣', '營業縣市_新竹市', '營業縣市_桃園市', '營業縣市_高雄市', '場所規模_4',
       '是否兼具製造業', '投保責任險'],
      dtype='object')

In [ ]:
# 違規案件預測結果排序
df_rsl = pd.DataFrame(
    {'真實值': y_test, '預測機率': lgs.predict_proba(X=X_test)[:, 1].round(4)}
)

i = 0
for id, real, prob in df_rsl.sort_values(by='預測機率', ascending=False).itertuples(): 
    i += 1
    if real: 
        print(f'第{id}筆資料為{real}，機率為{prob}，排序在第{i}位')


第6748筆資料為True，機率為0.0236，排序在第22位
第6644筆資料為True，機率為0.0101，排序在第105位
第6499筆資料為True，機率為0.0056，排序在第216位
第6450筆資料為True，機率為0.0019，排序在第702位
第6547筆資料為True，機率為0.0012，排序在第900位
第5954筆資料為True，機率為0.0009，排序在第976位


### stepwise AIC (LinAlgError: Singular matrix)

In [ ]:
import statsmodels.api as sm

In [ ]:

cols_inclu = list(X.columns)
best_aic = None

# while True:

aic_with_rmCol = []

for col in cols_inclu: 
    cols_try = list(set(cols_inclu)-{col})
    X_train = sm.add_constant(X[cols_try])
    mdl = sm.Logit(y, X_train).fit()
    aic = mdl.aic
    aic_with_rmCol.append((aic, col))

aic_with_rmCol.sort()
best_rmCol_aic, best_rmCol = aic_with_rmCol[0]

    # if best_aic is None or best_rmCol_aic < best_aic: 
    #     best_aic = best_rmCol_aic
    #     cols_inclu.remove(best_rmCol)

    #     print(f'移除特徵：{best_rmCol}，AIC = {best_rmCol_aic:.4f}')
    # else: 
    #     break

